In [85]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [5]:
fake = faker.Faker(locale='en')

In [6]:
amazon_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'Host': 'www.amazon.jobs',
    'content-type': 'application/json',
    'referer': 'https://www.amazon.jobs/en/search?base_query=&loc_query='
}

In [66]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

Здесь выбраны только FullTime и Part Time вакансии отсортированные по дате добавления (сначала более новые)

In [74]:
API_URL = "http://www.amazon.jobs/en/search.json?schedule_type_id[]=Flex-Time (<19 hours)&schedule_type_id[]=Part-Time (20-29 hours)&schedule_type_id[]=Reduced-Time (30-39 hours)&schedule_type_id[]=Flex Time (<19 hours)&schedule_type_id[]=Reduced Time (30-39 hours)&schedule_type_id[]=Full-Time&radius=24km&facets[]=normalized_country_code&facets[]=normalized_state_name&facets[]=normalized_city_name&facets[]=location&facets[]=business_category&facets[]=category&facets[]=schedule_type_id&facets[]=employee_class&facets[]=normalized_location&facets[]=job_function_id&facets[]=is_manager&facets[]=is_intern&sort=recent&latitude=&longitude=&loc_group_id=&loc_query=&base_query=&city=&country=&region=&county=&query_options=&"
MAIN_URL = "https://www.amazon.jobs"

### Тестирование для одной страницы

In [82]:
search_params = {
    'offset': 0,
    'result_limit': 100
}

In [83]:
result = requests.get(API_URL, headers=amazon_headers, params=search_params)
result

<Response [200]>

In [86]:
result.json()['jobs'][1]

{'basic_qualifications': '<ul><li>3+ years of design, implementation, or consulting experience in applications or infrastructures</li><li>8+ years within specific technology domain areas (e.g. software development, cloud computing, systems engineering, infrastructure, security, networking, data & analytics)</li></ul><br/>* Demonstrated ability to think strategically about technical challenges, business, and product<br/>* Strong verbal and written communication skills<br/>* Deep knowledge of IPv4 and IPv6 protocols and standards<br/>* Experience with routing protocols, especially BGP, and connectivity technologies like MPLS<br/>* Understanding of the building blocks and protocols used in SD-WAN technologies<br/>* Ability to travel up to 30% as needed',
 'business_category': 'aws',
 'city': 'Seattle',
 'company_name': 'Amazon Web Services, Inc.',
 'country_code': 'USA',
 'description': 'Job summary<br/>Come change the way networks are built in the cloud! AWS Networking powers the most se

### Все страницы

In [98]:
job_dicts = []
for page in tqdm(range(0, 100)): # max offset for amazon is 9999 :(
    search_params = {
        'offset': page * 100,
        'result_limit': 100
    }
    result = requests.get(API_URL, headers=amazon_headers, params=search_params).json()
    if result['jobs']:

        for job_info in result['jobs']:
            try:
                job_dict = {
                    'title': job_info['title'],
                    'internal_id' : job_info['id'],
                    'url': MAIN_URL + job_info['job_path'],
                    'description': clear_string(job_info['description']),
                    'responsibilities': None,
                    'qualifications': clear_string(job_info['basic_qualifications']),
                    'team': " ".join(job_info['team']['label'].split('-')) if job_info['team']['label'] else None,
                    'company': 'Amazon', #job_info['company_name'],
                    'publish_date': parser.parse(job_info['posted_date']),
                }
                job_dicts.append(job_dict)
            except:
                print(f'Error while parsing {MAIN_URL + job_info["job_path"]}')

    time.sleep(0.2) # sleep for 200 ms

 51%|█████     | 51/100 [01:23<01:21,  1.67s/it]

Error while parsing https://www.amazon.jobs/en/jobs/2120688/applied-science-manager-tts-research


 98%|█████████▊| 98/100 [02:44<00:03,  1.79s/it]

Error while parsing https://www.amazon.jobs/en/jobs/2108227/quality-assurance-engineer-ii-sft


100%|██████████| 100/100 [02:48<00:00,  1.68s/it]


In [99]:
snapshot = pd.DataFrame(job_dicts)
print(len(snapshot))
snapshot.sample(5)

9998


,title,internal_id,url,description,responsibilities,qualifications,team,company,publish_date
7715,"Account Manager - Mid Market, Western Canada, ...",03c15a59-cc7c-43e4-bfd3-3cb30c590b14,https://www.amazon.jobs/en/jobs/2114145/accoun...,Job summary Would you like to be part of a tea...,None,Basic Qualifications • Bachelor’s degree or eq...,team public sector,Amazon,2022-06-21
6816,Senior Data Engineer - AWS FinTech,4beb464d-38e6-42e6-81dd-0266492c7204,https://www.amazon.jobs/en/jobs/2116290/senior...,Job summary Are you passionate about data? Doe...,None,5+ years of experience as a Data Engineer or i...,team aws fintech,Amazon,2022-06-22
8521,"Support Ops Specialist, CN Support Ops",9c154648-0e1e-4884-b979-94682c6bf4ad,https://www.amazon.jobs/en/jobs/2112142/suppor...,Job summary Fulfillment By Amazon (FBA) is bui...,None,Good written and verbal communication skills i...,team global fulfillment loss prevention and se...,Amazon,2022-06-20
2156,"Network Development Engineer, Frontier Network...",98b0daea-61d2-4801-a1cb-cf026ff6f1c5,https://www.amazon.jobs/en/jobs/2127629/networ...,Job summary The AWS Networking organization is...,None,* Bachelor’s Degree in a technology related fi...,team no matching primary label listed,Amazon,2022-06-29
6351,"Support Engineer IV, DSP Site Reliability Engi...",7a207709-1e55-45a1-8674-110dbddb0df2,https://www.amazon.jobs/en/jobs/2117873/suppor...,Job summary Advertising is one of the hottest ...,None,3+ years of development or technical support e...,team amazondsp,Amazon,2022-06-23


In [100]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             9998 non-null   object        
 1   internal_id       9963 non-null   object        
 2   url               9998 non-null   object        
 3   description       9998 non-null   object        
 4   responsibilities  0 non-null      object        
 5   qualifications    9998 non-null   object        
 6   team              9972 non-null   object        
 7   company           9998 non-null   object        
 8   publish_date      9998 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(8)
memory usage: 703.1+ KB


In [101]:
snapshot['company'].value_counts()

Amazon    9998
Name: company, dtype: int64

In [102]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'02-07-2022'

In [103]:
snapshot.to_csv(f'../data/amazon/{current_date}.csv')